<a href="https://colab.research.google.com/github/nick-nagaraj/Santander-Kaggle/blob/master/Santander_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [6]:
!kaggle competitions download -c santander-customer-transaction-prediction

100% 122M/122M [00:03<00:00, 38.8MB/s]
100% 122M/122M [00:03<00:00, 42.4MB/s]
  0% 0.00/463k [00:00<?, ?B/s]
100% 463k/463k [00:00<00:00, 30.2MB/s]
 88% 107M/122M [00:02<00:00, 37.9MB/s] 
100% 122M/122M [00:02<00:00, 43.0MB/s]


In [7]:
!unzip '/content/train.csv.zip' 
!unzip '/content/test.csv.zip' 


Archive:  /content/train.csv.zip
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.csv                


In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
import pickle
from sklearn.preprocessing import MinMaxScaler
import csv
from sklearn import model_selection
from sklearn import metrics
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
from keras import models, layers, regularizers

def noise(X, y, n, sigma):
    _X = X.copy()
    _y = y.copy()
    for _ in range(n):
        X = np.r_[X, _X + np.random.randn(*_X.shape)*sigma]
        y = np.r_[y, _y]
    return X, y
 

In [3]:
data = pd.read_csv('train.csv', header = 0)
data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [0]:
cols = []
for i in range(0, 200):
  cols.append("var_{}".format(i))

In [0]:
y_train = data['target']
x_train = data[cols]
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=0)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
logreg = GradientBoostingClassifier(loss = "deviance", n_estimators=1200, learning_rate=0.3, max_depth=3, random_state=34, verbose = 2, validation_fraction = 0.35, subsample = 0.8).fit(X_train, Y_train)
filename = 'finalized_model.sav'
pickle.dump(logreg, open(filename, 'wb'))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.6374           0.0115           63.86m
         2           0.6266           0.0070           62.94m
         3           0.6194           0.0070           62.98m
         4           0.6156           0.0055           62.78m
         5           0.6102           0.0052           62.69m
         6           0.6027           0.0044           62.55m
         7           0.5964           0.0037           62.21m
         8           0.5927           0.0030           61.96m
         9           0.5883           0.0038           61.70m
        10           0.5818           0.0033           61.72m
        11           0.5751           0.0036           61.84m
        12           0.5741           0.0038           61.94m
        13           0.5707           0.0025           62.00m
        14           0.5647           0.0023           61.93m
        15           0.5614           0.0026           61.86m
       

In [41]:
X_test = scaler.fit_transform(X_test)
y_pred = logreg.predict(X_test)
print('Accuracy of classifier on test set: {:.2f}'.format(logreg.score(X_test, Y_test)))

Accuracy of logistic regression classifier on test set: 0.92


In [0]:
data_test = pd.read_csv('test.csv', header = 0)

x_test = data_test[cols]

x_test = scaler.fit_transform(x_test)
#x_test = preprocessing.normalize(x_test, norm='l1')

y_test = logreg.predict(x_test)




In [0]:
 
with open('final.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['ID_code','target'])
    for i in range(0,len(y_test)):
       val_1 = "test_{}".format(i)
       val_2 = y_test[i]
       filewriter.writerow([val_1,val_2])
       

In [53]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96     71876
           1       0.70      0.36      0.47      8124

   micro avg       0.92      0.92      0.92     80000
   macro avg       0.81      0.67      0.71     80000
weighted avg       0.91      0.92      0.91     80000



In [57]:
data_trial = pd.read_csv('final.csv')
data_trial.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0
